In [101]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pmdarima as pm
import keras.backend as K
from keras.layers import LSTM, Dense, Input, Bidirectional
from keras.optimizers import Adagrad
from keras.losses import mean_squared_logarithmic_error
from keras.models import load_model, Model
from keras.preprocessing.sequence import pad_sequences
from keras.constraints import nonneg
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [102]:
DATA_DIR = 'data'
OUTPUT_FILE = 'submission.csv'
TRAIN_SEQ_SIZE = 66
TEST_SEQ_SIZE = 43
N_IN = 6
N_VARS = 2
BATCH_SIZE = TRAIN_SEQ_SIZE - N_IN
EPOCHS = 10

def load_initial_data():
    train_path = os.path.join(DATA_DIR, 'train.csv')
    test_path = os.path.join(DATA_DIR, 'test.csv')
    train_data, test_data = pd.read_csv(train_path), pd.read_csv(test_path)
    return train_data, test_data

def load_extended_data():
    train_path = os.path.join(DATA_DIR, 'extended_train.csv')
    test_path = os.path.join(DATA_DIR, 'extended_test.csv')
    train_data, test_data = pd.read_csv(train_path), pd.read_csv(test_path)
    return train_data, test_data

In [103]:
def extract_days_from_first_infection(data: pd.DataFrame, write_file=False):
    result = list()
    prev_region, prev_country = None, None
    current_count = 0
    found_case = False
    for value in data.values:
        if prev_country is None:
            prev_region, prev_country = value[1], value[2]
        if value[1] is prev_region and value[2] == prev_country:
            if value[14] == 0:
                result.append(current_count)
            elif value[14] != 0 and not found_case:
                found_case = True
                result.append(current_count)
            elif value[14] != 0 and found_case:
                current_count += 1
                result.append(current_count)
        else:
            found_case = False
            current_count = 0
            if value[14] == 0:
                result.append(current_count)
            elif value[14] != 0:
                found_case = True
                result.append(current_count)
        prev_region, prev_country = value[1], value[2]
    data['Days since first infection'] = result
    if write_file:
        data.to_csv('added_days_train.csv', index=False)
    

def add_days_from_first_infection_test(train_data: pd.DataFrame, test_data: pd.DataFrame, write_file=False):
    max_counts = dict()
    result = list()
    for value in train_data.values:
        max_counts[str(value[1]) + str(value[2])] = value[-1]
    previous_key = None
    current_count = 0
    for value in test_data.values:
        key = str(value[1]) + str(value[2])
        if previous_key is None or key != previous_key:
            current_count = max_counts[key] + 1
        else:
            current_count += 1
        result.append(current_count)
        previous_key = key
    test_data['Days since first infection'] = result
    if write_file:
        test_data.to_csv('added_days_test.csv', index=False)

In [104]:
def replace_missing_extra_values_with_mean(data, write_file=False):
    data = data.mask(data == 0).fillna(data.mean())
    if write_file:
        data.to_excel('extra_features_improved.xlsx', index=False)

def merge_with_extra(train_df: pd.DataFrame, test_df: pd.DataFrame, extra_df: pd.DataFrame, write_file=False):
    train_df = train_df.merge(extra_df, how='left', on='Country_Region')
    test_df = test_df.merge(extra_df, how='left', on='Country_Region')
    if write_file:
        train_df.to_csv('extended_train_merged.csv', index=False)
        test_df.to_csv('extended_test_merged.csv', index=False)
    

In [105]:
def convert_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if isinstance(data, list) else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i))
                  for j in range(n_vars)]
    # output sequence
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1))
                      for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1))
                      for j in range(n_vars)]
    # combine all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [106]:
def root_mean_squared_log_error(y_true, y_predicted):
    return K.sqrt(mean_squared_logarithmic_error(y_true, y_predicted))


def build_model(X_train, y_train, load_path=None, validation_data=None,
                batch_size=BATCH_SIZE, epochs=EPOCHS):
    if load_path:
        model = load_model(load_path)
    else:
        inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
        lstm_1 = LSTM(units=40, activation='relu',
                      kernel_constraint=nonneg(),
                      recurrent_constraint=nonneg(),
                      bias_constraint=nonneg())(inputs)
        output = Dense(2, activation='relu',
                       kernel_constraint=nonneg(),
                       bias_constraint=nonneg())(lstm_1)
        model = Model(inputs=inputs, outputs=output)
        model.compile(loss='msle', optimizer='adam')
        model.summary()
        if validation_data is not None:
            X_test, y_test = validation_data
            history = model.fit(X_train, y_train, epochs=EPOCHS,
                                batch_size=batch_size,
                                validation_data=(X_test, y_test),
                                verbose=2, shuffle=False)
        else:
            history = model.fit(X_train, y_train, epochs=EPOCHS,
                                batch_size=batch_size,
                                verbose=2, shuffle=False)
        #plt.plot(history.history['loss'], label='train')
        #plt.plot(history.history['val_loss'], label='test')
        #plt.legend()
        #plt.show()
        model.save('model_v1.h5')
    return model

In [107]:
def predict_sequence_full(model, data, window_size, timesteps):
    # Shift the window by 1 new prediction each time,
    # re-run predictions on new window.
    curr_frame = data[0]
    predicted = []
    for i in range(timesteps):
        predicted.append(model.predict(curr_frame[np.newaxis,:,:])[0])
        curr_frame = curr_frame[1:]
        curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    return np.vstack(predicted)

def predict(model, train_df, eval_df, n_in=N_IN, n_vars=N_VARS,
            n_out=TEST_SEQ_SIZE, scaler=None):
    reframed_eval = {}
    for key, batch in train_df.groupby(['Country_Region', 'Province_State']):
        reframed_batch = convert_to_supervised(batch[['ConfirmedCases', 'Fatalities']],
                                               n_in=n_in)
        frame = reframed_batch.iloc[-1, n_vars:]
        reframed_eval[key] = frame.values
    predictions = []
    for key, _ in eval_df.groupby(['Country_Region', 'Province_State']):
        values = reframed_eval[key]
        X_eval = values.reshape((1, n_in, n_vars))
        predictions.append(predict_sequence_full(model, X_eval, n_in, n_out))
    predictionsdf = pd.DataFrame(columns=['Id', 'ConfirmedCases', 'Fatalities'])
    predictionsdf['Id'] = eval_df['ForecastId']
    predictionsdf[['ConfirmedCases', 'Fatalities']] = np.vstack(predictions)
    if scaler is not None:
        predictionsdf[['ConfirmedCases', 'Fatalities']] = scaler.inverse_transform(predictionsdf[['ConfirmedCases', 'Fatalities']])
    predictionsdf[['ConfirmedCases', 'Fatalities']] = predictionsdf[['ConfirmedCases', 'Fatalities']].astype(int)
    return predictionsdf

In [108]:
def get_features(df: pd.DataFrame, n_in=N_IN, n_vars=N_VARS,
                 train_split=None, batch_size=BATCH_SIZE, scaler=None):
    if scaler is not None:
        df.loc[:, ['ConfirmedCases', 'Fatalities']] = scaler.fit_transform(df[['ConfirmedCases', 'Fatalities']])
    reframed = []
    for key, batch in df.groupby(['Country_Region', 'Province_State']):
        reframed_batch = convert_to_supervised(batch[['ConfirmedCases', 'Fatalities']],
                                               n_in=n_in)
        reframed.append(reframed_batch)
    random.shuffle(reframed)
    reframed = pd.concat(reframed, ignore_index=True)
    values = reframed.values
    if train_split is not None:
        split_point = batch_size * int(train_split * (len(reframed) // batch_size))
        train, test = values[:split_point, :], values[split_point:, :]
        X_train, y_train = train[:, :-n_vars], train[:, -n_vars:]
        X_test, y_test = test[:, :-n_vars], test[:, -n_vars:]
        X_train = X_train.reshape((X_train.shape[0], n_in, n_vars))
        X_test = X_test.reshape((X_test.shape[0], n_in, n_vars))
        return (X_train, y_train), (X_test, y_test)
    else:
        X_train, y_train = values[:, :-n_vars], values[:, -n_vars:]
        X_train = X_train.reshape((X_train.shape[0], n_in, n_vars))
        return X_train, y_train


def get_regression_features(df:pd.DataFrame, for_train=True):
    df['Province_State'] = df['Province_State'].fillna('<placeholder>')
    results = [group for _, group in df.groupby(['Country_Region', 'Province_State'])]
    groups = np.stack(results)
    if for_train:
        feature_columns = [i for i in range(4, 20) if i not in [14, 15]]
        label_columns = [14, 15]
    else:
        feature_columns = [i for i in range(4, 18)]
        label_columns = None
    features = groups[:, :, feature_columns]
    if label_columns is not None:
        labels = groups[:, :, label_columns]
        return features, labels
    return features, None


def get_arima_features(df:pd.DataFrame, for_train=True):
    df['Province_State'] = df['Province_State'].fillna('<placeholder>')
    results = [group for _, group in df.groupby(['Country_Region', 'Province_State'])]
    groups = np.stack(results)
    if for_train:
        feature_columns = [i for i in range(4, 20) if i not in [14, 15]]
        label_columns = [14, 15]
    else:
        feature_columns = [i for i in range(4, 18)]
        label_columns = None
    features = groups[:, :, feature_columns]
    if label_columns is not None:
        labels_1 = groups[:, :, label_columns[0]]
        labels_2 = groups[:, :, label_columns[1]]
        return features, [labels_1, labels_2]
    return features, None

In [109]:
def create_nn_output_file(predictions, build_version=1):
    with open(OUTPUT_FILE, 'w') as f:
        f.write('ForecastId,ConfirmedCases,Fatalities\n')
        count = 1
        if build_version < 3:
            for pred in predictions:
                for i in range(TEST_SEQ_SIZE):
                    f.write(f'{str(count)},{str(int(pred[i][0]))},{str(int(pred[i][1]))}\n')
                    count += 1
        else:
            for first_pred, second_pred in zip(*predictions):
                for i in range(TEST_SEQ_SIZE):
                    f.write(f'{str(count)},{str(first_pred[i][0])},{str(second_pred[i][0])}\n')
                    count += 1


def create_regression_output_file(predictions):
    with open(OUTPUT_FILE, 'w') as f:
        f.write('ForecastId,ConfirmedCases,Fatalities\n')
        count = 1
        for country in predictions:
            for day in country:
                f.write(f'{str(count)},{str(int(day[0]))},{str(int(day[1]))}\n')
                count += 1


def create_arima_output_file(cases_predictions, fatalities_predictions):
    with open(OUTPUT_FILE, 'w') as f:
        f.write('ForecastId,ConfirmedCases,Fatalities\n')
        count = 1
        for prediction in zip(cases_predictions, fatalities_predictions):
            f.write(f'{str(count)},{str(int(prediction[0]))},{str(int(prediction[1]))}\n')
            count += 1


def normalize_input(train, test):
    # reshape for MinMaxScaler
    train_dims, test_dims = train.shape, test.shape
    train_features = train.reshape(train_dims[0], train_dims[1] * train_dims[2])
    test_features = test.reshape(test_dims[0], test_dims[1] * test_dims[2])
    
    normalizer = MinMaxScaler()
    normalizer = normalizer.fit(train_features)
    train_scaled = normalizer.transform(train_features)
    test_scaled = normalizer.transform(test_features)
    
    # reshape scaled back to 3 dimensions
    train_scaled = train_scaled.reshape(train_dims)
    test_scaled = test_scaled.reshape(test_dims)
    return train_scaled, test_scaled

In [110]:
def neural_main():
    labeled, unlabeled = load_extended_data()
    labeled['Province_State'] = labeled['Province_State'].fillna('<placeholder>')
    unlabeled['Province_State'] = unlabeled['Province_State'].fillna('<placeholder>')
    scaler = MinMaxScaler(feature_range=(0, 1))
    # 62 is the sequence length for train data and 43 for the testing data
    (X_train, y_train), (X_test, y_test) = get_features(labeled,
                                                        scaler=scaler,
                                                        train_split=0.8)
    model = build_model(X_train, y_train, validation_data=(X_test, y_test))
    predictionsdf = predict(model, labeled, unlabeled, scaler=scaler)
    predictionsdf.to_csv('data/nn_submission.csv', index=False)

In [111]:
def regression_main(use_exponential=False):
    train, test = load_extended_data()
    train_features, train_labels = get_regression_features(train)
    test_features, _ = get_regression_features(test, for_train=False)
    
    # for each country fit a separate model
    predictions = list()
    for country in range(train_features.shape[0]):
        clf = LinearRegression()
        if not use_exponential:
            clf.fit(train_features[country], train_labels[country])
            prediction = clf.predict(test_features[country])
        else:
            # TODO: Needs weights for a better prediction
            y_shape = train_labels[country].shape
            
            log_labels = [np.log(train_labels[country][i][j]) for i in range(y_shape[0]) 
                          for j in range(y_shape[1])]
            log_labels = np.array(log_labels)
            log_labels = np.nan_to_num(log_labels)
            # scipy Bug fix below!!!
            log_labels[log_labels < 0.00000001] = 0.00000001
            log_labels = np.reshape(log_labels, y_shape)

            clf.fit(train_features[country], log_labels)
            prediction = np.exp(clf.predict(test_features[country]))
        predictions.append(prediction)
    create_regression_output_file(predictions)


def arima_main():
    train, test = load_extended_data()
    train_features, train_labels = get_arima_features(train)
    test_features, _ = get_arima_features(test, for_train=False)
    
    # for each country fit a separate model
    cases_predictions = list()
    fatalities_predictions = list()
    for country in range(train_features.shape[0]):
        print(country)
        # Endogenous -> train_labels (ConfirmedCases + Fatalities)
        # Exogenous -> train_features (Everything we've collected)
        cases_model = pm.arima.AutoARIMA(start_p=1, start_q=1, max_p=7, max_q=7, maxiter=100,
                                         seasonal=False, suppress_warnings=True)
        cases_model.fit(train_labels[0][country], exogenous=train_features[country])
        cases_prediction = cases_model.predict(n_periods=test_features.shape[1], 
                                               exogenous=test_features[country])
        cases_predictions.extend(cases_prediction)
        fatalities_model = pm.arima.AutoARIMA(start_p=1, start_q=1, max_p=7, max_q=7, maxiter=100,
                                              seasonal=False, suppress_warnings=True)
        fatalities_model.fit(train_labels[1][country], exogenous=train_features[country])
        fatalities_prediction = fatalities_model.predict(n_periods=test_features.shape[1], 
                                                         exogenous=test_features[country])
        fatalities_predictions.extend(fatalities_prediction)
    create_arima_output_file(cases_predictions, fatalities_predictions)


neural_main()
# regression_main(use_exponential=False)
# arima_main()
# train, test = load_extended_data()
# extract_days_from_first_infection(train, True)
# add_days_from_first_infection_test(train, test, True)
# extra = pd.read_excel('data/extra_features.xlsx')
# merge_with_extra(train, test, extra, True)

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 6, 2)              0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 40)                6880      
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 82        
Total params: 6,962
Trainable params: 6,962
Non-trainable params: 0
_________________________________________________________________
Train on 14100 samples, validate on 3540 samples
Epoch 1/10
 - 1s - loss: 2.5990e-04 - val_loss: 1.4166e-06
Epoch 2/10
 - 1s - loss: 1.2565e-04 - val_loss: 1.1254e-06
Epoch 3/10
 - 1s - loss: 1.4335e-04 - val_loss: 1.2049e-06
Epoch 4/10
 - 1s - loss: 1.1596e-04 - val_loss: 1.0192e-06
Epoch 5/10
 - 1s - loss: 1.3390e-04 - val_loss: 1.1011e-06
Epoch 6/10
 - 1s - loss: 1.0926e-04 - val_l